In [16]:
import io
import os
import time
import json
from pymongo import MongoClient

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud import storage

In [2]:
mongo_client = MongoClient()
text_db = mongo_client.s2t

In [3]:
videos = mongo_client.text_db.videos

In [35]:
text_db.collection_names()

['videos']

In [5]:
# Instantiates a client
speech_client = speech.Client()
storage_client = storage.Client(project='speech-to-nlp')

In [6]:
try:
    bucket = storage_client.get_bucket('galen_interviews')
except google.cloud.exceptions.NotFound:
    print('Sorry, that bucket does not exist!')

In [7]:
iter_item = bucket.list_blobs()

In [8]:
for blob in iter_item:
    print(blob.media_link)

https://www.googleapis.com/download/storage/v1/b/galen_interviews/o/output.raw?generation=1488831342983557&alt=media


In [9]:
# Loads the audio into memory
with io.open('Amazon Coding Interview Question and Answer 1 Count Negative Integers in Matrix.mp3', 'rb') as audio_file:
    content = audio_file.read()
    audio_sample = speech_client.sample(
        source_uri='gs://galen_interviews/output.raw',
        encoding='LINEAR16',
        sample_rate=16000)

In [13]:
operation = speech_client.speech_api.async_recognize(audio_sample)

retry_count = 1000
while retry_count > 0 and not operation.complete:
    retry_count -= 1
    time.sleep(2)
    operation.poll()

if operation.complete:
    print('Operation complete!')
else:
    print('Failed.')

Operation complete!


In [20]:
alternatives = operation.results

In [31]:
text = ''
for alternative in alternatives:
    text+= alternative.transcript

In [32]:
text = {'text' : text}

In [33]:
text_db.videos.insert_one(text)

In [37]:
videos.